# B3LYP 能量与 XYG3 能量

在这一节中，我们会在闭壳层下给出 B3LYP 与 XYG3 能量；同时验证与 DFT 计算有关的矩阵，以及了解较为基础的 DFT 格点积分有关的代码书写．

In [ ]:
from pyscf import gto, scf, dft, mp, ao2mo, lib
import numpy as np

这一节我们还会对格点作简单的可视化．因此需要引入下面的工具．其中，`%matplotlib notebook` 是在 Jupyter Notebook 中嵌入交互式的图像的工具．

In [ ]:
# %matplotlib notebook

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

<div class="alert alert-info">

**提示**

这里把 `%matplotlib notebook` 注释掉了，但这只是因为生成该网页的 `nbsphinx` 似乎未必允许 GUI 形式的输出．如果你是用的是货真价实的 Jupyter Notebook，这个 Magic Command 对于 3D 图像的交互可视化确实是很有用的．

</div>

## XYG3 型双杂化泛函参考文献

XYG3 型双杂化泛函 (xDH, XYG3 type of Double Hybrid functional) 是一系列引入精确交换能与 PT2 形式交换能的泛函家族，其最初的泛函是 XYG3．其它典型的泛函有 XYGJ-OS、xDH-PBE0 等．

* XYG3
    * Zhang, Y.; Xu, X.; Goddard, W. A. Doubly Hybrid Density Functional for Accurate Descriptions of Nonbond Interactions, Thermochemistry, and Thermochemical Kinetics. *Proc. Natl. Acad. Sci. U.S.A.* **2009**, *106* (13), 4963–4968.
    * doi: [10.1073/pnas.0901093106](https://dx.doi.org/10.1073/pnas.0901093106)
* XYGJ-OS
    * Zhang, I. Y.; Xu, X.; Jung, Y.; Goddard, W. A. A Fast Doubly Hybrid Density Functional Method close to Chemical Accuracy Using a Local Opposite Spin Ansatz. *Proc. Natl. Acad. Sci. U.S.A.* **2011**, *108* (50), 19896–19900.
    * doi: [10.1073/pnas.1115123108](https://doi.org/10.1073/pnas.1115123108)
* xDH-PBE0
    * Zhang, I. Y.; Su, N. Q.; Brémond, É. A. G.; Adamo, C.; Xu, X. Doubly Hybrid Density Functional xDH-PBE0 from a Parameter-Free Global Hybrid Model PBE0. *J. Chem. Phys.* **2012**, *136* (17), 174103.
    * doi: [10.1063/1.3703893](https://doi.org/10.1063/1.3703893)

对 XYG3 泛函的一些测评、原理与展望等说明，有且不限于以下的文献：

* The XYG3 Type of Doubly Hybrid Density Functionals
    * Su, N. Q.; Xu, X. *WIREs Comput. Mol. Sci.* **2016**, *6* (6), 721–747.
    * doi: [10.1002/wcms.1274](https://doi.org/10.1002/wcms.1274)
* Development of New Density Functional Approximations
    * Su, N. Q.; Xu, X. *Annu. Rev. Phys. Chem.* **2017**, *68* (1), 155–182.
    * doi: [10.1146/annurev-physchem-052516-044835](https://doi.org/10.1146/annurev-physchem-052516-044835)
* Doubly Hybrid Density Functionals That Correctly Describe Both Density and Energy for Atoms
    * Su, N. Q.; Zhu, Z.; Xu, X. *Proc. Natl. Acad. Sci. U.S.A.* **2018**, *115* (10), 2287–2292.
    * doi: [10.1073/pnas.1713047115](https://doi.org/10.1073/pnas.1713047115)

## 预备工作

### 顶层函数计算 B3LYP 能量

由于我们准备与 Gaussian 核对能量，因此这里使用 VWN3 型的 LDA 相关泛函作为 B3LYP 中 LDA 相关泛函．

In [ ]:
mol = gto.Mole()
mol.atom = """
O  1.0  0.0  0.0
H  1.0  1.0  0.0
H  1.0  0.0  1.0
"""
mol.basis = "6-31G"
mol.build()

In [ ]:
scf_eng = dft.RKS(mol)
scf_eng.xc = "b3lypg"  # compare that to gaussian
scf_eng.grids.atom_grid = (99, 590)
scf_eng.grids.build()
scf_eng.conv_tol = 1e-13
energy_scf = scf_eng.kernel()

### B3LYP 轨道构造的 MP2 相关能

我们可以直接将上述的 B3LYP 轨道代入 MP2 相关能公式中．求得的 MP2 相关能将会是 XYG3 能量构成的一部分．在这里我们可以预先生成之．

In [ ]:
mp2_eng = mp.MP2(scf_eng)
energy_mp2_corr, _ = mp2_eng.kernel()

### B3LYP 重要中间矩阵

在上一份文档中，我们已经对众多 HF 中间矩阵作了较为充分的说明．在这里，我们就简单地将这些重要矩阵写在一起．

In [ ]:
nao = mol.nao
nmo = scf_eng.mo_energy.shape[0]
nelec = mol.nelectron
nocc = mol.nelec[0]
nvir = nmo - nocc

S = mol.intor('int1e_ovlp_sph')
T = mol.intor('int1e_kin_sph')
Vnuc = mol.intor('int1e_nuc_sph')
eri = mol.intor('int2e_sph')

C = scf_eng.mo_coeff
Co = C[:, :nocc]
Cv = C[:, nocc:]
e = scf_eng.mo_energy
eo = e[:nocc]
ev = e[nocc:]

D = scf_eng.make_rdm1()
F = scf_eng.get_fock()

J = scf_eng.get_j()
K = scf_eng.get_k()

通常来说，由于 HF 与 B3LYP 的计算方式几乎一样，因此上一份文档中提及的大多数性质都能保证；但关于 Fock 矩阵的验证上，需要注意到只有一部分性质仍然存在：

In [ ]:
print("Eigenvalues of Fock   :", np.allclose(C.T @ F @ C, np.eye(nmo) * e))
print("Fock matrix decompose :", np.allclose(T + Vnuc + J - 0.5 * K, F))

这也就意味着对于 DFT 方法，我们仍然需要了解 Fock 矩阵的构建方式．对于能量亦是如此：

In [ ]:
np.allclose((D * (T + Vnuc + 0.5 * J - 0.25 * K)).sum() + mol.energy_nuc(), \
            scf_eng.energy_tot())

下面一节中，我们就将解决如何求取 Fock 矩阵与 B3LYP 能量的问题．

## B3LYP 交换相关势与交换相关能

### 通过 PySCF 高级函数生成

我们首先通过 PySCF 较为顶层的函数来生成交换相关势 $V^\mathrm{xc}_{\mu \nu} [\mathbf{D}]$ 与交换相关能 $E^\mathrm{xc} [\mathbf{D}]$．这两者分别可以通过 Fock 矩阵与 B3LYP 能量来验证．下面的代码会一次性地生成这两者，以及对密度矩阵 $D_{\mu \nu}$ 的电子数的数值求和：

In [ ]:
xc_n, xc_e, xc_v = \
    scf_eng._numint.nr_rks(mol, scf_eng.grids, scf_eng.xc, D)

这个函数的主体是 `scf_eng._numint`，它是 `dft.numint.Numint` 类型；我们可以直接调用这个类型的函数进行计算，其结果是相同的：

In [ ]:
xc_n_ni, xc_e_ni, xc_v_ni = \
    dft.numint.nr_rks(scf_eng._numint, mol, scf_eng.grids, scf_eng.xc, D)
print(np.allclose(xc_n_ni, xc_n))
print(np.allclose(xc_e_ni, xc_e))
print(np.allclose(xc_v_ni, xc_v))

该函数的传入参数分别是分子构型、格点信息、泛函信息以及 AO 密度矩阵．

### 输出 1：电子数和

我们知道，水分子的电子数是 10 个．该函数的第一个输出即可以验证电子数是否合理．

In [ ]:
print(xc_n)
np.allclose(xc_n, nelec)

### 输出 2：交换相关能

第二个输出是交换相关能．获得该能量后，我们可以验证 B3LYP 总能量了．但在此之前，我们需要先知道 B3LYP 作为杂化泛函，其杂化 HF 型交换能，即交换积分的比例系数 $c_\mathrm{x}$．这里我们使用下面的方法导出系数．

In [ ]:
c_x = scf_eng._numint.hybrid_coeff(scf_eng.xc)

随后我们可以通过杂化泛函的计算通式给出 B3LYP 的总能量．这里同时列出杂化泛函的计算式与 HF 的计算式，相以比对：

\begin{align}
E^\textrm{Hyb} &= D_{\mu \nu} (T_{\mu} + V_{\mu}^\mathrm{Nuc} + \frac{1}{2} J_{\mu \nu} [\mathbf{D}] - \frac{1}{4} c_x K_{\mu \nu} [\mathbf{D}]) + E^\mathrm{xc} [\mathbf{D}] +  E^\mathrm{Nuc} \\
E^\textsf{HF} &= D_{\mu \nu} (T_{\mu} + V_{\mu}^\mathrm{Nuc} + \frac{1}{2} J_{\mu \nu} [\mathbf{D}] - \frac{1}{4} K_{\mu \nu} [\mathbf{D}]) +  E^\mathrm{Nuc}
\end{align}

In [ ]:
np.allclose((D * (T + Vnuc + 0.5 * J - 0.25 * c_x * K)).sum() + xc_e + mol.energy_nuc(), \
            scf_eng.energy_tot())

<div class="alert alert-info">

**提示**

一般地，在程序中对 DFT 部分的计算与对交换积分的计算是分开的；因此，从实现的角度上讲，$E^\mathrm{xc} [\mathbf{D}]$ 不包含交换积分．

</div>

### 输出 3：交换相关势

类似于交换相关能，当获得交换相关势后，我们就可以验证 Fock 矩阵．我们也对杂化泛函与 HF 的 Fock 矩阵作比对：

\begin{align}
F_{\mu \nu}^\textrm{Hyb} &= T_{\mu \nu} + V_{\mu \nu}^\mathrm{Nuc} + J_{\mu \nu} [\mathbf{D}] - \frac{1}{2} c_x K_{\mu \nu} [\mathbf{D}] + V_{\mu \nu}^\mathrm{xc} [\mathbf{D}] \\
F_{\mu \nu}^\textsf{HF} &= T_{\mu \nu} + V_{\mu \nu}^\mathrm{Nuc} + J_{\mu \nu} [\mathbf{D}] - \frac{1}{2} K_{\mu \nu} [\mathbf{D}]
\end{align}

In [ ]:
np.allclose(T + Vnuc + J - 0.5 * c_x * K + xc_v, F)

## 格点积分方法

### DFT 计算使用的格点

上面使用了顶层的 `dft.numint.NumInt.nr_rks` 函数生成了计算 B3LYP 能量时所需要的交换相关势 $V^\mathrm{xc}_{\mu \nu} [\mathbf{D}]$ 与交换相关能 $E^\mathrm{xc} [\mathbf{D}]$；但该函数比较高级．如果我们需要作一些底层的修改，该函数就不合适了．为此，我们会初步地对其中的底层实现作基础的了解．

DFT 的矩阵与能量的计算关键是格点积分部分．之后的几小节将会在假定格点信息已经具备的情况下，通过对格点数据的求和来得到我们期望的交换相关势与能量．

在这一小节中，我们仅仅是研究格点信息，因此会使用非常小的格点数据进行讨论．我们所选取的格点大小是径向 4 个点，角向 14 个点的格点积分．

In [ ]:
grid_test = dft.gen_grid.Grids(mol)
grid_test.atom_grid = (4, 14)
grid_test.build()

我们知道，得到 DFT 的交换相关势相当于需要泛函对密度或动能密度的一阶梯度，因此这里设置梯度量为 1．通过 PySCF 的中层函数，我们可以给出目前格点下所需要的众多信息：

In [ ]:
ao_deriv = 1
grid_ao, grid_mask, grid_weight, grid_coords = \
    next(scf_eng._numint.block_loop(mol, grid_test, nao, ao_deriv, 2000))
print("Shape of arrays:")
print("ao     :", grid_ao.shape)
print("mask   :", grid_mask.shape)
print("weight :", grid_weight.shape)
print("coords :", grid_coords.shape)

其中，`grid_coords` 变量代表格点的坐标，`grid_weight` 变量代表对应坐标的格点权重．我们可以使用下面的代码进行可视化．注意到因为一部分格点的权重为零，因此在作对数图可视化时需要加一个小量 (在实际进行积分时则不需要引入)；这里假的小量为 $10^{-5}$．这一般来说不影响我们对格点数据的定性观察．

In [ ]:
weight_shifted = grid_weight + 1e-5
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(grid_coords.T[0], grid_coords.T[1], grid_coords.T[2], \
                c=weight_shifted, \
                norm=LogNorm(vmin=weight_shifted.min(), vmax=weight_shifted.max()))
fig.colorbar(sc)
# fig.show()

关于变量 `grid_ao` 的意义，以及 `grid_weight` 的意义，将会在下面两小节中进行说明．

刚才的格点 `grid_test` 数量显然太少，只有 $4 \times 14 = 168$ 个点．在实际的量化计算中，显然这是不合适的．我们现在回到真实环境下所使用的格点 (99, 590)，来进行后续的计算．

对于变量 `grid_mask`，猜测为因为在一些原子轨道积分 `grid_ao` 下，值总是为零或非常小，于是为了加速格点求和，忽略这些格点积分的数值．在这份笔记中，我们暂时不需要关心这些为加速运算速度而引入的数组．

In [ ]:
ao_deriv = 1
grid_ao, grid_mask, grid_weight, grid_coords = \
    next(scf_eng._numint.block_loop(mol, scf_eng.grids, nao, ao_deriv, 2000))
print("Shape of arrays:")
print("ao     :", grid_ao.shape)
print("mask   :", grid_mask.shape)
print("weight :", grid_weight.shape)
print("coords :", grid_coords.shape)
ngrids = grid_weight.shape[0]

### 输出 1：电子数和格点求取，格点积分实现

首先，我们会使用格点积分处理最简单的电子数的和．这里的记号会混用 Einstein Convention 与积分记号，因此只会在引入格点积分时出现求和记号，其它时候不引入；符号上多少会复杂一些，但应当不会产生歧义．

我们知道，电子密度的和可以用下述的方式给出：($D_{ij}$ 与 $D_{\mu \nu}$ 分别是 MO 基组与 AO 基组密度矩阵，它们两者不同但有下述的关系)

\begin{align}
n &= D_{ij} = 2 \delta_{ij} = \mathrm{tr} (\mathbf{D}^\mathrm{MO}) \\
&= C_{\mu i} S_{\mu \nu} C_{\nu j} = \mathrm{tr} (\mathbf{C}^\mathrm{occ, T} \mathbf{S} \mathbf{C}^\mathrm{occ}) = \mathrm{tr} (\mathbf{C}^\mathrm{occ, T} \mathbf{C}^\mathrm{occ} \mathbf{S}) = \mathrm{tr} (\mathbf{D}^\mathrm{AO} \mathbf{S}) \\
&= (C_{\mu i} C_{\nu j} \delta_{ij}) S_{\mu \nu} = D_{\mu \nu} S_{\mu \nu}
\end{align}

下面我们只验证 $n = D_{\mu \nu} S_{\mu \nu}$：

In [ ]:
np.allclose((D * S).sum(), nelec)

上式完全是矩阵的运算．但如果我们希望使用格点积分，对空间变量 $\boldsymbol{r}$ 显式地积分，那么上式可以根据重叠矩阵的定义写为

\begin{equation}
n = D_{\mu \nu} \int \phi_{\mu} (\boldsymbol{r}) \phi_{\nu} (\boldsymbol{r}) \, \mathrm{d} \boldsymbol{r}
\end{equation}

随后我们就可以进行格点积分计算了．格点积分的实现方式是对空间坐标积分量离散化并继而求和．具体地来说，如果现在的被积函数是 $f(\boldsymbol{r})$，那么对于一系列已知坐标的、角标记为 $g$ 的格点 (即坐标集合) $\{ \boldsymbol{r}_g \}_g$ 与格点权重 (即实数集合) $\{ w_g \}_g$，有

\begin{equation}
\int f(\boldsymbol{r}) \mathrm{d} \boldsymbol{r} \simeq \sum_{g} f(\boldsymbol{r}_g) w_g
\end{equation}

如果现在将 $f(\boldsymbol{r}_g)$ 记为 $f_g$，即将函数 $f(\boldsymbol{r})$ 映射到格点集合 $\{ f_g \}_g$，那么我们可以使用 Einstein Convention 简记上述积分为

\begin{equation}
\int f(\boldsymbol{r}) \mathrm{d} \boldsymbol{r} \simeq f_g w_g
\end{equation}

回到电子数的格点积分．如果我们定义对原子轨道基组的格点映射为 $\phi_{\mu} (\boldsymbol{r}) \mapsto \{ \phi_{g \mu} \}_g$，那么电子数积分就可以写为

\begin{equation}
n = D_{\mu \nu} \phi_{g \mu} \phi_{g \nu} w_g
\end{equation}

下面我们就通过格点积分验证电子数的和．这里指出，[上文中](#DFT-格点) `block_loop` 函数所声称的 `grid_ao` 是四部分构成的，其中的第一部分 `grid_ao[0]` 是原子轨道基组的格点 $\phi_{g \mu}$，而剩下三部分 `grid_ao[1:4]` 则是原子轨道基组的 $(x, y, z)$ 梯度分量 $\nabla_t \phi_{g \mu}$，其中 $t \in (x, y, z)$．在电子数求和时只需要 $\phi_{g \mu}$ 即可，而 $\nabla_t \phi_{g \mu}$ 在交换相关势与能量求取是会使用．

In [ ]:
xc_n_sum = float(np.einsum("uv, gu, gv, g", D, grid_ao[0], grid_ao[0], \
                           grid_weight, optimize=True))
xc_n_sum

In [ ]:
np.allclose(xc_n_sum, xc_n)

我们甚至可以顺便验证 $S_{\mu \nu} = \phi_{g \mu} \phi_{g \nu} w_g$，不过因为格点积分的精度不算太高，因此在使用 `np.allclose` 验证矩阵相同时，需要把默认精度 $10^{-8}$ 降低到 $10^{-7}$.

In [ ]:
np.allclose(np.einsum("gu, gv, g", grid_ao[0], grid_ao[0], \
                      grid_weight, optimize=True), \
            S, atol=1e-7)

### 输出 2：交换相关能格点求取

交换相关能从定义上，其积分表达式为

\begin{equation}
E^\textrm{xc} [\rho] = \int f^\textrm{xc} [\rho, \nabla \rho] \rho(\boldsymbol{r}) \, \mathrm{d} \boldsymbol{r}
\end{equation}

显然地，上式经过格点化后可以写为

\begin{equation}
E^\textrm{xc} [\rho] = f^\textrm{xc}_g \rho_g w_g
\end{equation}

看起来非常简洁．但是上式的 $f^\textrm{xc}_g$ 与 $\rho_g$ 都还不是已知量．$\rho_g$ 的构造方法比较容易：

\begin{equation}
\rho_g = \rho (\boldsymbol{r}_g) = D_{\mu \nu} \phi_{\mu} (\boldsymbol{r}_g) \phi_{\nu} (\boldsymbol{r}_g) = D_{\mu \nu} \phi_{g \mu} \phi_{g \nu}
\end{equation}

为了构造 $f^\textrm{xc}_g$，我们需要传入关于 $\rho, \boldsymbol{\nabla} \rho$ 的信息，并且了解作为 DFT 近似核心的 $f^\textrm{xc}_g$．PySCF 中，这些计算已经通过接口函数 `eval_xc` 完成，其底层一般来说是 LibXC 库函数．我们需要准备的信息不多，只需要下面三行代码即足够：

In [ ]:
grid_rho = np.einsum("uv, tgu, gv -> tg", D, grid_ao, grid_ao[0], optimize=True)
grid_rho[1:4] *= 2
grid_exc, grid_vxc = scf_eng._numint.eval_xc(scf_eng.xc, grid_rho, deriv=1)[:2]

关于上面三行代码，我们还有不少需要说明的地方．首先，关于第一行，实际上是合并了两个操作，同时导出了 $\rho_g$ (`grid_rho[0]`) 与 $\nabla_t \rho_g$ (`grid_rho[1:4]`)．$\rho_g$ 的导出方式上面已经叙述了；而 $\nabla_t \rho_g$ 的导出方式如下：

\begin{align}
\nabla_t \rho_g &= \nabla_t \rho (\boldsymbol{r}_g) = D_{\mu \nu} \nabla_t \big( \phi_{\mu} (\boldsymbol{r}_g) \phi_{\nu} (\boldsymbol{r}_g) \big) \\
&= D_{\mu \nu} \nabla_t \phi_{\mu} (\boldsymbol{r}_g) \phi_{\nu} (\boldsymbol{r}_g) + D_{\mu \nu} \phi_{\mu} (\boldsymbol{r}_g) \nabla_t \phi_{\nu} (\boldsymbol{r}_g) \\
&= D_{\mu \nu} (\nabla_t \phi_{g \mu}) \phi_{g \nu} + D_{\mu \nu} \phi_{g \mu} (\nabla_t \phi_{g \nu}) \\
&= 2 D_{\mu \nu} (\nabla_t \phi_{g \mu}) \phi_{g \nu}
\end{align}

联系到 `grid_ao[1:4]` 储存了 $\nabla_t \phi_{g \mu}$，而 `grid_ao[0]` 储存的是 $\phi_{g \mu}$，应当不难理解上述代码的第一行为何可以同时导出 $\rho_g$ 和 $\nabla_t \rho_g$ 了．但是我们看到上面公式中有两倍，但 $\nabla_t \rho_g$ 公式中只有一倍，因此我们有必要对 `grid_ao[1:4]` 部分单独乘上 2 倍．

第三行代码则是具体执行计算 $f^\mathrm{xc}_g$ 的代码．根据该代码的 [API 文档](https://sunqm.github.io/pyscf/dft.html#pyscf.dft.libxc.eval_xc)，由于我们不仅需要计算泛函的能量本身，还要计算其梯度以构造 Fock 矩阵，因此设置 `deriv=1`；由此，`eval_xc` 函数的输出尽管是一个四元素 tuple，但只有前两个元素非 `None`，且分别为泛函能量格点 `grid_exc` 与梯度格点 `grid_vxc`；后两个元素本来应当是泛函二阶梯度与三阶梯度量，但在 GGA 的 SCF 中不需要计算它们．关于梯度格点，将在下一小节中说明．

我们在构造 `grid_rho[0]` 后，我们还可以先简单验证其可以导出电子数之和对于水分子应当为 10：

\begin{equation}
n = \rho_g w_g
\end{equation}

In [ ]:
(grid_rho[0] * grid_weight).sum()

能量值的导出也相当容易：我们可以发现，`grid_exc` 的维度就是格点大小：

In [ ]:
grid_exc.shape

那么只要简单地套到能量格点公式 $E^\textrm{xc} [\rho] = f^\textrm{xc}_g \rho_g w_g$ 中，就可以给出交换相关能．我们验证这一结果．

In [ ]:
xc_e_sum = (grid_exc * grid_rho[0] * grid_weight).sum()
xc_e_sum

In [ ]:
np.allclose(xc_e_sum, xc_e)

### 输出 3：交换相关势格点求取

交换相关势的求取公式仍然不算复杂．在这一小节，我们先列出公式与代码，再对它们作较为详细的解释；下一小节中，我们会简单地推导交换相关势的计算公式．

交换相关势的计算公式为

\begin{align}
V^\textrm{xc}_{\mu \nu} [\rho] 
&= w_g (\partial_\rho f^\mathrm{xc}_{g}) \phi_{g \mu} \phi_{g \nu} \\
&\quad + 2 w_g (\partial_\gamma f^\mathrm{xc}_{g}) (\nabla_t \rho_g) (\nabla_t \phi_{g \mu}) \phi_{g \nu} \\
&\quad + 2 w_g (\partial_\gamma f^\mathrm{xc}_{g}) (\nabla_t \rho_g) (\nabla_t \phi_{g \nu}) \phi_{g \mu}
\end{align}

In [ ]:
grid_frho, grid_fgamma = grid_vxc[:2]

xc_v_sum = np.einsum("g, g, gu, gv -> uv", grid_weight, grid_frho, \
                     grid_ao[0], grid_ao[0], optimize=True)
xc_v_sum += 2 * np.einsum("g, g, tg, tgu, gv -> uv", grid_weight, grid_fgamma, \
                          grid_rho[1:4], grid_ao[1:4], grid_ao[0], optimize=True)
xc_v_sum += 2 * np.einsum("g, g, tg, tgv, gu -> uv", grid_weight, grid_fgamma, \
                          grid_rho[1:4], grid_ao[1:4], grid_ao[0], optimize=True)

我们可以验证上面导出的交换相关势 `xc_v_sum` 与使用高级函数 `nr_rks` 所导出的 `xc_v` 是相同的：

In [ ]:
np.allclose(xc_v_sum, xc_v)

上述代码中需要解释的是第一行．`grid_vxc` 是 `eval_xc` 函数所导出的第二个变量．我们知道 `eval_xc` 会返回四个变量，分别是关于能量格点、一阶导数格点、二阶导数格点、三阶导数格点．那么 `grid_vxc` 所表征的是一阶导数格点．

尽管我们现在接触的是以 GGA 为底层的 B3LYP，但这里有必要简单了解 meta-GGA．meta-GGA 的泛函变量一般至多是下述四个数量值的几种：

\begin{equation}
\rho, \quad 
\gamma = \boldsymbol{\nabla} \rho \cdot \boldsymbol{\nabla} \rho, \quad
\boldsymbol{\nabla}^2 \rho, \quad
\tau = | \boldsymbol{\nabla} \phi_\mu |^2
\end{equation}

因此，在程序导出泛函的梯度时，也会相应地导出

\begin{equation}
\left(
\frac{\partial f^\mathrm{xc}}{\partial \rho}, 
\frac{\partial f^\mathrm{xc}}{\partial \gamma}, 
\frac{\partial f^\mathrm{xc}}{\partial (\boldsymbol{\nabla}^2 \rho)}, 
\frac{\partial f^\mathrm{xc}}{\partial \tau}
\right)
\end{equation}

上面的写法不适合得到文档的行内表示，因此后面会简记上面的记号为

\begin{equation}
(
\partial_\rho f^\mathrm{xc},
\partial_\gamma f^\mathrm{xc},
\partial_{\boldsymbol{\nabla}^2 \rho} f^\mathrm{xc},
\partial_\tau f^\mathrm{xc}
)
\end{equation}

上面写出的函数偏导实际上是关于空间变量 $\boldsymbol{r}$ 的函数，只是因为记号复杂而将其省略．如果我们写得详细一些，譬如 GGA Kernel 下的 $\partial_\rho f^\mathrm{xc}$，应该表示为

\begin{equation}
(\partial_\rho f^\mathrm{xc}) (\boldsymbol{r}) = \frac{\partial f^\mathrm{xc} (\rho, \gamma)}{\partial \rho (\boldsymbol{r})}
\end{equation}

当要化为格点时，即取 $\boldsymbol{r}$ 为确定的坐标 $\boldsymbol{r}_g$ 时，简记下述记号

\begin{equation}
\partial_\rho f^\mathrm{xc}_g = (\partial_\rho f^\mathrm{xc}) (\boldsymbol{r}_g)
\end{equation}

对于 $\partial_\gamma f^\mathrm{xc}_g$ 亦如此．这样，我们就解释了上面代码所对应的公式的记号了．

对于 RKS 而言，`grid_vxc` 变量确实就是四维 tuple 变量；因此，第一行代码

```
grid_frho, grid_fgamma = grid_vxc[:2]
```

所作的就是将 $\partial_\rho f^\mathrm{xc}_g$ 赋值给 `grid_frho`；$\partial_\gamma f^\mathrm{xc}_g$ 赋值给 `grid_fgamma`；这些变量的维度都是格点大小．由于 B3LYP 是 GGA，因此 $\partial_{\boldsymbol{\nabla}^2 \rho} f^\mathrm{xc}, \partial_\tau f^\mathrm{xc}$ 没有意义，即 `grid_vxc[2:4]` 不应是有意义的量．下面的代码可以简单说明这些．

In [ ]:
print(grid_vxc.__len__())
print(grid_vxc[2:4])
print(grid_frho.shape)
print(grid_fgamma.shape)

<div class="alert alert-info">

**提示**

但对于 UKS 而言，由于 $\alpha$ 与 $\beta$ 密度不同，因此 `eval_xc` 给出的一阶导数格点尽管仍然是四维 tuple 变量，但矩阵的数量总共有 9 个．更多信息需要参考 [API 文档](https://sunqm.github.io/pyscf/dft.html#pyscf.dft.libxc.eval_xc)．

</div>

### 交换相关势的推导

这里的推导仅仅是表明交换相关势中每一项、以及它们的系数是如何导出的；更为详细与严谨的推导还需要参考其它文献．

泛函的变分一般来说只有放在积分的环境下才具有意义．根据 [Wikipedia](https://en.wikipedia.org/wiki/Functional_derivative#Formula) 的说明，如果能量泛函记为

\begin{equation}
E^\textrm{xc} [\rho] = \int f^\textrm{xc} (\rho, \gamma) \rho(\boldsymbol{r}) \, \mathrm{d} \boldsymbol{r}
\end{equation}

我们可以对 GGA 型的泛函一次变分记为

\begin{align}
\int \frac{\delta E^\textrm{xc} [\rho]}{\delta \rho (\boldsymbol{r})} \phi (\boldsymbol{r}) \, \mathrm{d} \boldsymbol{r} 
&= \int \left[ \frac{\partial f^\textrm{xc} (\rho, \gamma)}{\partial \rho (\boldsymbol{r})} \phi (\boldsymbol{r})
+ \frac{\partial f^\textrm{xc} (\rho, \gamma)}{\partial \boldsymbol{\nabla}_{\boldsymbol{r}'} \rho (\boldsymbol{r})} \cdot \boldsymbol{\nabla}_{\boldsymbol{r}'} \phi (\boldsymbol{r}) \right] 
\, \mathrm{d} \boldsymbol{r} \\
&= \int \left[ \frac{\partial f^\textrm{xc} (\rho, \gamma)}{\partial \rho (\boldsymbol{r})} \phi (\boldsymbol{r})
+ \frac{\partial f^\textrm{xc} (\rho, \gamma)}{\partial \gamma (\boldsymbol{r})} 2 \boldsymbol{\nabla}_{\boldsymbol{r}'} \rho (\boldsymbol{r}) \cdot \boldsymbol{\nabla}_{\boldsymbol{r}'} \phi (\boldsymbol{r}) \right] 
\, \mathrm{d} \boldsymbol{r}
\end{align}

第一个等式是变分本身的定义；第二个等式利用到连续偏导以及关于 $\gamma$ 与 $\boldsymbol{\nabla}_{\boldsymbol{r}'} \rho$ 关系的一个性质：

\begin{equation}
\frac{\partial \gamma (\boldsymbol{\nabla}_{\boldsymbol{r}'} \rho)}{\partial \boldsymbol{\nabla}_{\boldsymbol{r}'} \rho (\boldsymbol{r})}
= \frac{\partial \boldsymbol{\nabla}_{\boldsymbol{r}'} \rho \cdot \boldsymbol{\nabla}_{\boldsymbol{r}'} \rho}{\partial \boldsymbol{\nabla}_{\boldsymbol{r}'} \rho (\boldsymbol{r})}
= 2 \boldsymbol{\nabla}_{\boldsymbol{r}'} \rho (\boldsymbol{r})
\end{equation}

上面出现记号 $\boldsymbol{\nabla}_{\boldsymbol{r}'}$；这仅仅是普通的梯度记号，一般会记为 $\nabla$；这里记得较为复杂，只是强调梯度的坐标变量 $\boldsymbol{r}'$ 与被积元变量 $\boldsymbol{r}$ 不是相同的．如果我们使用 Einstein Convention，对 $\boldsymbol{r}'$ 更换为 $t \in (x, y, z)$ 并在实际计算过程中对 $t$ 求和，来记 GGA 的一次变分；同时按照 [上一小节](#输出-3：交换相关势格点求取) 简化偏导数记号；那么可以写为

\begin{equation}
\int \frac{\delta E^\textrm{xc} [\rho]}{\delta \rho (\boldsymbol{r})} \phi (\boldsymbol{r}) \, \mathrm{d} \boldsymbol{r}
= \int \left[ (\partial_{\rho} f^\mathrm{xc}) \phi + 2 (\partial_{\gamma} f^\mathrm{xc}) (\nabla_t \rho) (\nabla_t \phi) \right] \, \mathrm{d} \boldsymbol{r}
\end{equation}

现在我们考察上述结论是如何应用到 $V^\textrm{xc}_{\mu \nu} [\rho]$ 的导出过程中的．我们知道，$V^\textrm{xc}_{\mu \nu} [\rho]$ 是 Fock 矩阵构成的一部分；Fock 矩阵的构成方式是将 Hartree-Fock 能量对分子轨道进行变分而产生的．能量可以分为单电子、库伦积分、交换积分与泛函积分的贡献，那么 Fock 矩阵也会分为这四部分；关于这点我们已经在 [上文](#输出-3：交换相关势) 中验证过了．那么泛函积分对 Fock 矩阵的贡献可以表示为

\begin{equation}
F_{\mu \nu} [\rho] \leftarrow V^\textrm{xc}_{\mu \nu} [\rho] = \int \frac{\delta E^\textrm{xc} [\rho]}{\delta \phi_{\mu} (\boldsymbol{r})} \phi_{\nu} (\boldsymbol{r}) \, \mathrm{d} \boldsymbol{r}
\end{equation}

由于密度的表达式可以写为 (该公式不使用 Einstein Convention)

\begin{equation}
\rho(\boldsymbol{r}) = \rho [\{ \phi_{\mu} \}_\mu] (\boldsymbol{r}) = \int \sum_{\nu} \phi_{\nu} (\boldsymbol{r}) \phi_{\nu} (\boldsymbol{r}') \delta(\boldsymbol{r} - \boldsymbol{r}') \, \mathrm{d} \boldsymbol{r}'
\end{equation}

因此密度对轨道的变分可以写作 (该公式不使用 Einstein Convention)

\begin{align}
\frac{\delta \rho}{\delta \phi_\mu (\boldsymbol{r})} 
&= \int \sum_{\nu} \frac{\partial \phi_{\nu}}{\partial \phi_{\mu} (\boldsymbol{r})} \phi_{\nu} (\boldsymbol{r}') \delta(\boldsymbol{r} - \boldsymbol{r}') \, \mathrm{d} \boldsymbol{r}' \\
&= \int \phi_{\mu} (\boldsymbol{r}') \delta(\boldsymbol{r} - \boldsymbol{r}') \, \mathrm{d} \boldsymbol{r}' = \phi_\mu (\boldsymbol{r})
\end{align}

因此，根据连续变分规则，有
\begin{equation}
V^\textrm{xc}_{\mu \nu} [\rho] = \int \frac{\delta E^\textrm{xc} [\rho]}{\delta \rho (\boldsymbol{r})} \frac{\delta \rho}{\delta \phi_\mu (\boldsymbol{r})} \phi_{\nu} (\boldsymbol{r}) \, \mathrm{d} \boldsymbol{r}
= \int \frac{\delta E^\textrm{xc} [\rho]}{\delta \rho (\boldsymbol{r})} \phi_\mu (\boldsymbol{r}) \phi_{\nu} (\boldsymbol{r}) \, \mathrm{d} \boldsymbol{r}
\end{equation}

那么我们用上面已经有的结论，令 $\phi (\boldsymbol{r}) = \phi_\mu (\boldsymbol{r}) \phi_{\nu} (\boldsymbol{r})$，并注意到

\begin{equation}
\nabla_t \phi = \nabla_t (\phi_\mu \phi_\nu) = (\nabla_t \phi_\mu) \phi_\nu + (\nabla_t \phi_\nu) \phi_\mu
\end{equation}

那么

\begin{equation}
V^\textrm{xc}_{\mu \nu} [\rho] = \int \left[ (\partial_{\rho} f^\mathrm{xc}) \phi_\mu \phi_\nu + 2 (\partial_{\gamma} f^\mathrm{xc}) (\nabla_t \rho) (\nabla_t \phi_\mu) \phi_\nu + 2 (\partial_{\gamma} f^\mathrm{xc}) (\nabla_t \rho) (\nabla_t \phi_\nu) \phi_\mu \right] \, \mathrm{d} \boldsymbol{r}
\end{equation}

将上式格点化，就立即得到 [上一小节](#输出-3：交换相关势格点求取) 所使用的格点公式了．

## XYG3 能量

### 手动设置 B3LYP 泛函形式

XYG3 作为非自洽泛函，其密度与轨道从 B3LYP 获得，但其非自洽能量泛函则使用自己的泛函形式．如果要在尚未实现 XYG3 的软件中获得 XYG3 能量，就必须手动设置泛函参数．在实际使用 XYG3 参数前，我们先尝试对 B3LYP 进行手动的参数设置，并验证结果是否与上面的计算相同．

In [ ]:
b3try_eng = dft.RKS(mol)
b3try_eng.xc = "HF*0.2 + .08*LDA + .72*B88, .81*LYP + .19*VWN3"  # compare that to gaussian
b3try_eng.grids.atom_grid = (99, 590)
b3try_eng.grids.build()

b3try_eng.kernel()
print("Compare My B3LYPG: ", np.allclose(b3try_eng.e_tot, scf_eng.e_tot))

### XYG3 泛函形式与能量

XYG3 型双杂化泛函属于第五阶泛函 (按 Predew 的 [Jacob 阶梯](https://dx.doi.org/10.1063/1.1390175) 说法)，其 [定义](https://dx.doi.org/10.1073/pnas.0901093106) 是

\begin{equation}
E_\mathrm{xc}^\textsf{R5} = E_\mathrm{xc}^\textsf{LDA} + c_1 (E_\mathrm{x}^\textsf{exact} - E_\mathrm{x}^\textsf{LDA}) + c_2 \Delta E_\mathrm{x}^\textsf{GGA} + c_3 (E_\mathrm{c}^\textsf{PT2} - E_\mathrm{c}^\textsf{LDA}) + c_4 \Delta E_\mathrm{c}^\textsf{GGA}
\end{equation}

对应到程序中，每一项的系数则展开为

\begin{equation}
E_\mathrm{xc}^\textsf{R5} = (1 - c_1 - c_2) E_\mathrm{x}^\textsf{LDA} + c_2 E_\mathrm{x}^\textsf{GGA} + (1 - c_3 - c_4) E_\mathrm{c}^\textsf{LDA} + c_4 E_\mathrm{c}^\textsf{GGA} + c_1 E_\mathrm{x}^\textsf{exact} + c_3 E_\mathrm{c}^\textsf{PT2}
\end{equation}

对于 XYG3，其系数的确定是

\begin{equation}
c_1 = 0.8033, \quad c_2 = 0.2107, \quad c_3 = 0.3211, \quad c_4 = 1 - c_3
\end{equation}

因此，XYG3 的泛函形式为

\begin{equation}
E_\mathrm{xc}^\textsf{XYG3} = 0.8033 E_\mathrm{x}^\textsf{exact} - 0.0140 E_\mathrm{x}^\textsf{LDA} + 0.2107 E_\mathrm{x}^\textsf{GGA} + 0.6789 E_\mathrm{c}^\textsf{GGA} + 0.3211 E_\mathrm{c}^\textsf{PT2}
\end{equation}

在 PySCF 中，泛函形式的确定通过字符串进行定义．该字符串的定义方式可以很灵活，下面的代码只是其中一种定义方式．其中，逗号前后分别分割了交换能与相关能的部分．由于 PySCF 中不对 PT2 型相关能在 DFT 部分进行定义，因此在定义泛函的时候，不将 $0.3211 E_\mathrm{c}^\textsf{PT2}$ 写入字符串．

当泛函的形式确定后，我们只要简单地把自洽场过程中收敛的 B3LYP 密度代入 XYG3 型泛函能量表达式中，即可得到最终的 XYG3 能量了．

In [ ]:
nscf_eng = dft.RKS(mol)
nscf_eng.xc = "0.8033*HF - 0.0140*LDA + 0.2107*B88, 0.6789*LYP"
nscf_eng.grids.atom_grid = (99, 590)
nscf_eng.grids.build()

xyg3_energy = nscf_eng.energy_tot(dm=D) + mp2_eng.e_corr * 0.3211
print("XYG3 energy allclose: ", np.allclose(xyg3_energy, -0.76282393305943E+02))